In [2]:
file_path = 'API_KEY.txt'

with open(file_path, 'r') as file:
        api_key = file.read()

In [5]:
import os

main_drive_folder_path = "G:\Mijn Drive\Kassabonnen"

# get all file names of receipts in New folder -> receipts that need to be extracted. 
file_names = os.listdir(f"{main_drive_folder_path}/fotos/supermarkt")
print(file_names)

['20241222_172556.jpg', '20241223_214033.jpg', 'IMG-20241228-WA0000.jpg', '20241227_122135.jpg', '20241227_121112.jpg', '20241227_115048.jpg', 'IMG-20241227-WA0002.jpg', 'IMG-20241226-WA0025.jpg', '20250102_110922(0).jpg', 'desktop.ini']


In [39]:
import base64
import requests
import json
import re
import pandas as pd
import datetime

# functions

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  

prompt = """Geef informatie over de aankopen op de bon terug. Dit is een voorbeeld van de output, volg het zelfde format:
{"winkel": "Vomar",
    "producten": [
    {
        "beschrijving": "Wokkels Naturel",
        "aantal": 1,
        "totaal prijs": 2.69,
        "originele prijs per stuk": 2.69
    },
    {
        "beschrijving": "Unox Groente Soep",
        "aantal": 3,
        "totaal prijs": 4.8,
        "originele prijs per stuk": 1.60

    }
    ],
    "totaal prijs": 11.03,
    "totaal aantal producten": 4,
    "datum": "20-12-2024",
    "tijd": 13:34,
    "winkel categorie": "Supermarkt"}

Per product, geef het totaal bedrag en geef alleen het totaal bedrag met korting er van af.
"""
  
# prompt chatgpt
def prompt_chatgpt(base64_image):
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 2000
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  json_response = response.json()
  txt_response = json_response['choices'][0]['message']['content']
  return txt_response


def extract_csv_from_response(response):
    # remove unnecesary character before and after {}
    response = re.search(r'\{.*\}', response, re.DOTALL).group()
    # convert string into json
    data = json.loads(response)

    # convert json into df
    df = pd.DataFrame(data['producten'])
    df['winkel'] = data['winkel']
    df['datum'] = data['datum']
    df['tijd'] = data['tijd']
    df['winkel categorie'] = data['winkel categorie']
    df['prijs per product*'] = round(df['totaal prijs']/df['aantal'],2)
    df = df[['winkel', 'datum', 'tijd', 'beschrijving', 'prijs per product*', "originele prijs per stuk",'aantal', 'totaal prijs', 'winkel categorie']]

    total = {'prijs': data['totaal prijs'], 'aantal': data['totaal aantal producten']}
    return df, total



def create_overview_csv(df_products, elapsed_time, response, total):
    row = df_products.iloc[0]
    tijd = row['tijd'].replace(":", "-")
    id = f"{row['winkel']}{row['datum']}{tijd}"
    data = {'winkel':[row['winkel']],
            'datum': [row['datum']],
            'tijd': [row['tijd']],
            'totaal prijs': [total['prijs']],
            'totaal aantal': [total['aantal']],
            'winkel categorie': [row['winkel categorie']],
            'id': [id],
            'time_chatgpt_response':elapsed_time,
            'date extracted': datetime.date.today(),
            'response':response
            }
    
    df_overview = pd.DataFrame(data)
    return df_overview, id



In [19]:
image_vomar = encode_image(f"{main_drive_folder_path}/fotos/supermarkt/{file_names[0]}")
response_vomar = prompt_chatgpt(image_vomar)


In [20]:
image_lidl = encode_image(f"{main_drive_folder_path}/fotos/supermarkt/{file_names[1]}")
response_lidl = prompt_chatgpt(image_lidl)


In [45]:
image_ah = encode_image("G:\Mijn Drive\Kassabonnen/fotos\supermarkt\IMG-20241226-WA0025.jpg")
response_ah = prompt_chatgpt(image_ah)

In [ ]:
df_ah, tot_ah = extract_csv_from_response(response_ah)

In [40]:
df_vomar, tot_vomar = extract_csv_from_response(response_vomar)

In [41]:
df_lidl, tot_lidl = extract_csv_from_response(response_lidl)

In [47]:
display(df_ah)

,winkel,datum,tijd,beschrijving,prijs per product*,originele prijs per stuk,aantal,totaal prijs,winkel categorie
0,Albert Heijn,23-12-2024,21:51,AH Bami,3.99,3.99,1,3.99,Supermarkt
1,Albert Heijn,23-12-2024,21:51,Sate Ajam,1.29,1.29,2,2.58,Supermarkt


In [42]:
display(df_vomar)

,winkel,datum,tijd,beschrijving,prijs per product*,originele prijs per stuk,aantal,totaal prijs,winkel categorie
0,Vomar,02-12-2024,12:14,Wokkels Naturel,2.15,2.15,1,2.15,Supermarkt
1,Vomar,02-12-2024,12:14,Kalkoenfilet,2.89,2.89,1,2.89,Supermarkt
2,Vomar,02-12-2024,12:14,Dizi Panini,2.29,2.29,1,2.29,Supermarkt
3,Vomar,02-12-2024,12:14,Vici Surimi Sticks,1.36,1.36,1,1.36,Supermarkt


In [43]:
display(df_lidl)

,winkel,datum,tijd,beschrijving,prijs per product*,originele prijs per stuk,aantal,totaal prijs,winkel categorie
0,Lidl,23-12-2024,14:46,Zonnenbloemolie,4.49,4.49,2,8.98,Supermarkt
1,Lidl,23-12-2024,14:46,Pastasaus Napoli,2.98,2.98,1,2.98,Supermarkt
2,Lidl,23-12-2024,14:46,Picco pizza,2.49,2.49,1,2.49,Supermarkt
3,Lidl,23-12-2024,14:46,Halfvol kruidenmelk,0.87,0.87,4,3.49,Supermarkt
4,Lidl,23-12-2024,14:46,Bonenrolletjes in spek,2.39,2.39,1,2.39,Supermarkt
5,Lidl,23-12-2024,14:46,Feta kaas,1.69,1.69,1,1.69,Supermarkt
6,Lidl,23-12-2024,14:46,Melkchocolade,0.78,0.78,1,0.78,Supermarkt
7,Lidl,23-12-2024,14:46,Witte chocolade,1.19,1.19,1,1.19,Supermarkt
8,Lidl,23-12-2024,14:46,Pure chocolade,0.89,0.89,1,0.89,Supermarkt
9,Lidl,23-12-2024,14:46,Slagroom houdbaar,1.99,1.99,1,1.99,Supermarkt


In [6]:
import shutil
import time

# for each foto of receipt extract information.
for receipt in file_names:
    # encode each foto
    image_path = f"{folder_path}/{receipt}"
    base64_image = encode_image(image_path)

    # prompt chatgpt to extract information from receipt
    start_time = time.time()
    response = prompt_chatgpt(base64_image)
    elapsed_time = time.time() - start_time

    df_products, total = extract_csv_from_response(response)
    display(df_products)

    df_overview, id = create_overview_csv(df_products, elapsed_time, response, total)


    # save and update csv file
    if os.path.exists("extracted_data/records_products_csv.csv"):
        previous_records = pd.read_csv("extracted_data/records_products_csv.csv")
        df_products = pd.concat([previous_records, df_products])
    
    df_products.to_csv("extracted_data/records_products_csv.csv", index=False)


    if os.path.exists("extracted_data/overview_csv.csv"):
        previous_records = pd.read_csv("extracted_data/overview_csv.csv")
        df_overview = pd.concat([previous_records, df_overview])
    
    df_overview.to_csv("extracted_data/overview_csv.csv", index=False)

    shutil.move(image_path, f"fotos/Done/{id}.jpg")


,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Gamma,04-12-2024,10:15,Montagetape TESA,6.49,1,6.49,Bouwmarkt
1,Gamma,04-12-2024,10:15,Fijnstofmasker Ventiel Tegen Geur,22.99,1,22.99,Bouwmarkt


,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Hema,07-11-2024,13:49,String (3 stuks),2.86,3,8.59,Kleding
1,Hema,07-11-2024,13:49,"String (4 voor 24,99)",24.99,1,24.99,Kleding
2,Hema,07-11-2024,13:49,"Onderged 4 voor 24,99",2.33,1,2.33,Kleding


,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Kruidvat,19-12-2024,16:36,Kerstkaarten,2.99,2,5.98,Drogerij


,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Lidl,23-12-2024,14:46,Zonnebloemolie,2.49,2,4.98,Supermarkt
1,Lidl,23-12-2024,14:46,Pastasaus Napoli,2.98,1,2.98,Supermarkt
2,Lidl,23-12-2024,14:46,Picco pizza,2.99,1,2.99,Supermarkt
3,Lidl,23-12-2024,14:46,Halvolle zuivelmelk,0.99,4,3.96,Supermarkt
4,Lidl,23-12-2024,14:46,Bonnenrolletje in spek,2.29,1,2.29,Supermarkt
5,Lidl,23-12-2024,14:46,Feta kaas,1.69,1,1.69,Supermarkt
6,Lidl,23-12-2024,14:46,Melkchocolade,0.78,1,0.78,Supermarkt
7,Lidl,23-12-2024,14:46,Witte chocolade,1.19,1,1.19,Supermarkt
8,Lidl,23-12-2024,14:46,Pure chocolade,0.89,1,0.89,Supermarkt
9,Lidl,23-12-2024,14:46,Slagroom houdbaar,1.79,1,1.79,Supermarkt


,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Vomar,02-12-2024,12:14,Wokkels Naturel,2.15,1,2.15,Supermarkt
1,Vomar,02-12-2024,12:14,Kipfilet,2.89,1,2.89,Supermarkt
2,Vomar,02-12-2024,12:14,Panini,2.29,1,2.29,Supermarkt
3,Vomar,02-12-2024,12:14,Surimi Sticks,1.36,1,1.36,Supermarkt


In [14]:
display(df_products)

,winkel,datum,tijd,beschrijving,prijs per product*,aantal,totaal prijs,winkel categorie
0,Lidl,23-12-2024,14:46,Zonnebloemolie,4.49,2,8.98,Supermarkt
1,Lidl,23-12-2024,14:46,Pastaus Napoli,2.98,1,2.98,Supermarkt
2,Lidl,23-12-2024,14:46,Picco pizza,2.98,1,2.98,Supermarkt
3,Lidl,23-12-2024,14:46,Halfvolle melkdrink,0.87,4,3.49,Supermarkt
4,Lidl,23-12-2024,14:46,Bonnenrolletje in spek,1.89,1,1.89,Supermarkt
5,Lidl,23-12-2024,14:46,Feta kaas,1.29,1,1.29,Supermarkt
6,Lidl,23-12-2024,14:46,Melkchocolade,0.78,1,0.78,Supermarkt
7,Lidl,23-12-2024,14:46,Witte chocolade,1.19,1,1.19,Supermarkt
8,Lidl,23-12-2024,14:46,Pure chocolade,0.89,1,0.89,Supermarkt
9,Lidl,23-12-2024,14:46,Slagroom houdbaar,1.49,1,1.49,Supermarkt


In [15]:
display(df_overview)

,winkel,datum,tijd,totaal prijs,totaal aantal,winkel categorie,id,time_chatgpt_response,date extracted,response
0,Lidl,23-12-2024,14:46,69.63,29,Supermarkt,Lidl23-12-202414-46,33.789302,2024-12-23,Hier is de informatie over de aankopen op de b...
